In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

## Keras Initialization amd Model Preparation

In [3]:
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Activation, Permute
from keras.layers import LSTM, Dropout, CuDNNGRU, Bidirectional
from keras.utils import to_categorical, plot_model
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop, Adam, Adagrad, Adadelta, SGD
from keras.layers import Input, Flatten, Dense, Dropout, Convolution1D, Conv1D, Conv2D, \
                         MaxPool1D, MaxPool2D, Lambda, GlobalMaxPooling1D, \
                         GlobalAveragePooling1D, GlobalAveragePooling2D, \
                         BatchNormalization, Activation, AveragePooling1D, \
                         Concatenate, concatenate, Reshape, Embedding

from sklearn.model_selection import train_test_split, cross_val_score, \
                                    KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from keras.utils import np_utils

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

Using TensorFlow backend.


In [4]:
with K.tf.device('/GPU:0'):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1.
    session = tf.Session(config=config)
    K.set_session(session)

In [5]:
import os
import sys

import random
random.seed(42)

# os.environ["CUDA_VISIBLE_DEVICES"]="0"
curruser = os.environ.get('USER')
sys.path.insert(0, './src/')

import re
import string
from string import digits
import pandas as pd
import numpy as np
import joblib
import csv

from LSTMPreProc import preprocIO
from Bio import SeqIO
import smart_open
# from bpe import Encoder
import sentencepiece as spm
# from tokenizers import CharBPETokenizer, ByteLevelBPETokenizer, SentencePieceBPETokenizer
from tqdm import tqdm_notebook

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from pylab import rcParams
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 12, 5

In [6]:
from gensim.models.fasttext import FastText as FT_gensim
from gensim.corpora import Dictionary
from gensim import corpora
from gensim.models import TfidfModel, KeyedVectors
from gensim.test.utils import datapath
from gensim.utils import tokenize
from gensim.test.utils import common_texts

In [ ]:
# dnaSegPath = ('./data/dnaseg/mart_export.txt')
# prep = preprocIO()
# seqArr = []
# for seq in tqdm_notebook(prep.streamFastaIO(dnaSegPath)):
#      seqArr+=[seq]

In [7]:
def plot_metrics(history):
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    
    plt.figure()
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    
    
def confusion_matrix_heatmap(y_true, y_pred):
    '''
    Построение Confusion matrix (матрицы ошибок)

    Parameters
    ----------
    y_test: pandas.Series, numpy.array
        Целевая для обучающего набора
    y_pred: pandas.Series, numpy.array
        Значения целевой переменной, предсказанные классификатором
    '''
    rcParams['figure.figsize'] = 6, 4
    sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cmap='Greens')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()    

## Load Pretrained BPE Tokenizer 

In [8]:
sp = spm.SentencePieceProcessor()
sp.Load("models/BPE/sentpbpe.model")

True

## Generate Embedding Matrix: `(Token's Id, Weighted Vector)`

In [ ]:
import gzip
from collections import OrderedDict
import json

In [ ]:
def weight_words2vec(pathtofile, pathtoexport, sp, fasttext_model):
    corpora = []
    result = []  
    print('Tokenize sequense corpora from file-->')
    path = datapath(pathtofile)  # absolute path to corpus
    with open(path, 'r', encoding='utf8', errors = 'ignore') as fin:
        for line in tqdm_notebook(fin):
            line = line.rstrip('\n')
            corpora.append(list(sp.EncodeAsPieces(line))[1:])
    
    maxlen = max(map(len, corpora))
    
    print("max tokens len: {}".format(maxlen))
    
    print("Build BOW and calculate TFIDF...")
    dct = Dictionary(corpora)
    bow = [dct.doc2bow(line) for line in corpora]
    modelTfIdf = TfidfModel(bow, smartirs='ntc')
    dic = [(k,v) for k,v in dct.items()]
    rev_dct = dict(map(reversed, dic))
    
    del dct, dic
    
    print("Calculate Weighted Vector for each token of interest-->")
    for i,line in tqdm_notebook(enumerate(corpora), total=len(corpora)):
        if len(line)!=0:
            embed = []
            weights = dict(modelTfIdf[bow[i]])
            embed = [fasttext_model[word]*weights[rev_dct[word]] for word in line]
#             mean_vec = np.array(embed).mean(axis=0)
            result.append(embed)
        else:
            pass
#             mean_vec = np.zeros((model_gensim.vector_size,))

#         if not np.mod(len(result), 10000) and len(result)!=0:
#             with gzip.open(pathtoexport, 'at') as fz:
#             with open(str(pathtoexport),'a', encoding='utf8') as fz: 
#                 writer = csv.writer(fz, delimiter=';', quotechar='"', lineterminator='\n')
#                 file_is_empty = os.stat(str(pathtoexport)).st_size == 0
#                 if file_is_empty:
#                     writer.writerow(['idx','vec'])
#                 for k,val in result.items():
#                     strarr = ['{0:.8f}'.format(ele) for ele in val]
#                     fz.write('{}: '.format(k)+' '.join(strarr)+'\n')
#                     writer.writerow([k,v])

#             result = OrderedDict()
    
    return maxlen, result

## Calculate Weighted Embeddings ( This aproach is suitable only for the case of ad-hoc preparation of X matrix before feeding one on the input of Keras model)

In [9]:
dnaSegPathTrain = os.path.join(os.getcwd(),'data/dnaseg/csv/DnaSeg4TrainwLabelsSampled_wRandL.csv')
pathtoexport    = os.path.join(os.getcwd(),'models/embed/bpe_embeddings.json')

In [ ]:
model_gensim = joblib.load('./models/fasttext/ft_model.pkl')

In [ ]:
maxlen, result = weight_words2vec(dnaSegPathTrain, pathtoexport, sp, model_gensim)

In [ ]:
word_seq_pad = sequence.pad_sequences(result, maxlen = maxlen, padding='post')

## Generate Embeding Matrix: Get FT Embeddings without TFIDF weighting

In [9]:
fw2vec = './models/fasttext/ft_300.vec'
w2vec = KeyedVectors.load_word2vec_format(fw2vec)

In [ ]:
# Embed = w2vec.get_keras_embedding(train_embeddings=False)

In [10]:
embedding_matrix = np.zeros((sp.GetPieceSize()+1, w2vec.vector_size))
for i, vec in enumerate(w2vec.wv.vectors):
    embedding_matrix[sp.PieceToId(w2vec.index2word[i])] = vec

In [11]:
embedding_matrix.shape

(151, 300)

In [9]:
dnaSegPathTrain = os.path.join(os.getcwd(),'data/dnaseg/csv/DnaSeg4TrainwLabelsSampled_wRandL.csv')
data = pd.read_csv(dnaSegPathTrain, sep=';')

In [12]:
data.head()

,seq,label
0,GGACTGTGTGTGCAAAGGCGGAGAGGCAGCTGAGGCAGAAGCAGGA...,0
1,TGGAGCTGTCTCGCCTGGCCCGGATTGCGGACACCAAGATGAAATC...,0
2,ACCAAAAGATATTGGATTCCGACTCGACTCATTACATCCATCCTGC...,0
3,ATCTTCGTGGATGATGAGACGAAGTTGACGCTGCATGGGTTGCAGC...,0
4,ATACTGTCTTGCCATTTCCCTGCCTCTTTGACACTTATGTAGAAGT...,0


### Prepare 1-st Input: Create one-hot encoded vectors using the `char_indices` map

In [10]:
SEQUENCE_LENGTH = data.seq.str.len().max()

sentences = data.seq.tolist()
next_chars = data.label.tolist()

chars = sorted(list(set(sentences[0])))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)),dtype=np.bool)
# y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in tqdm_notebook(enumerate(sentences), total=len(sentences)):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
        #y[i, char_indices[next_chars[i]]] = 1

In [11]:
y = to_categorical(next_chars, num_classes=4)

In [ ]:
X.shape, y.shape

### Prepare 2-nd Input: Tensor with Tokens' Id to be passed on Embed Layer

In [12]:
def tokens_to_ids(encoder, x):
    x_toks = []
    for i in tqdm_notebook(range(len(x)), total=len(x)):
#         encoder.tokenize(x[i])
        x_toks.append(encoder.EncodeAsIds(x[i])[1:])
    
    max_len = max(map(len, x_toks))
    X_arr = np.zeros((len(x_toks), max_len), dtype='int32')
    for i, s in tqdm_notebook(enumerate(x_toks), total=len(x_toks)):
        X_arr[i, 0:len(s)] = s
    
    return X_arr

X_toks = tokens_to_ids(encoder=sp, x=data.seq.values)

In [13]:
X_toks = X_toks[:,:45]

In [14]:
X_toks_train, X_toks_test, y_train, y_test = train_test_split(X_toks, y, random_state=42, shuffle=True, test_size=0.1)

In [ ]:
# X_toks_train, X_toks_test, y_train, y_test = train_test_split(X_toks, y, random_state=42, shuffle=True, test_size=0.05)
# X_train, X_test, _, _ = train_test_split(X, y, random_state=42, shuffle=True, test_size=0.05)


In [ ]:
del X_toks, X

## Build Inception-ResNet Network

In [15]:
def conv2d(x,numfilt,filtsz,strides=1,pad='same',act=True,name=None):
    x = Conv2D(numfilt,filtsz,strides=strides,padding=pad,
               data_format='channels_last',use_bias=False,name=name+'conv2d')(x)
    x = BatchNormalization(axis=3,scale=False,name=name+'conv2d'+'bn')(x)
    if act:
        x = Activation('relu',name=name+'conv2d'+'act')(x)
    return x

### Inception ResNet A block

In [16]:
def incresA(x,scale,name=None):
    pad = 'same'
    branch0 = conv2d(x,8,1,1,pad,True,name=name+'b0')
    branch1 = conv2d(x,8,1,1,pad,True,name=name+'b1_1')
    branch1 = conv2d(branch1,32,3,1,pad,True,name=name+'b1_2')
    branch2 = conv2d(x,8,1,1,pad,True,name=name+'b2_1')
    branch2 = conv2d(branch2,12,3,1,pad,True,name=name+'b2_2')
    branch2 = conv2d(branch2,16,3,1,pad,True,name=name+'b2_3')
    branches = [branch0,branch1,branch2]
    mixed = Concatenate(axis=3, name=name + '_concat')(branches)
    filt_exp_1x1 = conv2d(mixed,96,1,1,pad,False,name=name+'filt_exp_1x1')
    final_lay = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
                       output_shape=K.int_shape(x)[1:],
                       arguments={'scale': scale},
                       name=name+'act_scaling')([x, filt_exp_1x1])
    return final_lay

### Inception ResNet B block#

In [17]:
def incresB(x,scale,name=None):
    pad = 'same'
    branch0 = conv2d(x,48,1,1,pad,True,name=name+'b0')
    branch1 = conv2d(x,32,1,1,pad,True,name=name+'b1_1')
    branch1 = conv2d(branch1,40,[1,7],1,pad,True,name=name+'b1_2')
    branch1 = conv2d(branch1,48,[7,1],1,pad,True,name=name+'b1_3')
    branches = [branch0,branch1]
    mixed = Concatenate(axis=3, name=name + '_mixed')(branches)
    filt_exp_1x1 = conv2d(mixed,288,1,1,pad,False,name=name+'filt_exp_1x1')
    final_lay = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
                      output_shape=K.int_shape(x)[1:],
                      arguments={'scale': scale},
                      name=name+'act_scaling')([x, filt_exp_1x1])
    return final_lay

### Inception ResNet C block

In [18]:
def incresC(x,scale,name=None):
    pad = 'same'
    branch0 = conv2d(x,48,1,1,pad,True,name=name+'b0')
    branch1 = conv2d(x,48,1,1,pad,True,name=name+'b1_1')
    branch1 = conv2d(branch1,56,[1,3],1,pad,True,name=name+'b1_2')
    branch1 = conv2d(branch1,64,[3,1],1,pad,True,name=name+'b1_3')
    branches = [branch0,branch1]
    mixed = Concatenate(axis=3, name=name + '_mixed')(branches)
    filt_exp_1x1 = conv2d(mixed,512,1,1,pad,False,name=name+'fin1x1')
    final_lay = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
                      output_shape=K.int_shape(x)[1:],
                      arguments={'scale': scale},
                      name=name+'act_saling')([x, filt_exp_1x1])
    return final_lay

In [19]:
def get_model(vocab_size, emb_vec_size, repeat, n_classes, seq_len_emb, seq_len_chars=None):

    ip  = Input(shape =(seq_len_emb,), name='Input1')
    emb = Embedding(vocab_size, emb_vec_size, name='BPE_emb')(ip)
    rep = Lambda(lambda inputs: K.repeat_elements(inputs,rep=repeat,axis=2),
                 name="repeat_elems")(emb)
    pre = Reshape((K.int_shape(rep)[1],emb_vec_size,repeat))(rep)      

    # Stem block

    x = conv2d(pre,8,2,2,'valid',True,name='conv1')
    x = conv2d(x,8,2,1,'valid',True,name='conv2')
    x = conv2d(x,16,2,1,'valid',True,name='conv3')

    x_11 = MaxPool2D(2,strides=1,padding='valid',name='stem_br_11'+'_maxpool_1')(x)
    x_12 = conv2d(x,16,2,1,'valid',True,name='stem_br_12')

    x = Concatenate(axis=3, name = 'stem_concat_1')([x_11,x_12])

    x_21 = conv2d(x,16,1,1,'same',True,name='stem_br_211')
    x_21 = conv2d(x_21,16,[1,7],1,'same',True,name='stem_br_212')
    x_21 = conv2d(x_21,16,[7,1],1,'same',True,name='stem_br_213')
    x_21 = conv2d(x_21,24,2,1,'valid',True,name='stem_br_214')

    x_22 = conv2d(x,16,1,1,'same',True,name='stem_br_221')
    x_22 = conv2d(x_22,24,2,1,'valid',True,name='stem_br_222')

    x = Concatenate(axis=3, name = 'stem_concat_2')([x_21,x_22])

    x_31 = conv2d(x,48,2,1,'valid',True,name='stem_br_31')
    x_32 = MaxPool2D(2,strides=1,padding='valid',name='stem_br_32'+'_maxpool_2')(x)
    x = Concatenate(axis=3, name = 'stem_concat_3')([x_31,x_32])    

    #Inception-ResNet-A modules
    x = incresA(x,0.15,name='incresA_1')
    x = incresA(x,0.15,name='incresA_2')
    x = incresA(x,0.15,name='incresA_3')

    #35 × 35 to 17 × 17 reduction module.
    x_red_11 = MaxPool2D(3,strides=2,padding='valid',name='red_maxpool_1')(x)
    x_red_12 = conv2d(x,96,3,2,'valid',True,name='x_red1_c1')

    x_red_13 = conv2d(x,64,1,1,'same',True,name='x_red1_c2_1')
    x_red_13 = conv2d(x_red_13,64,3,1,'same',True,name='x_red1_c2_2')
    x_red_13 = conv2d(x_red_13,96,3,2,'valid',True,name='x_red1_c2_3')

    x = Concatenate(axis=3, name='red_concat_1')([x_red_11,x_red_12,x_red_13])

    #Inception-ResNet-B modules
    x = incresB(x,0.1,name='incresB_1')
    x = incresB(x,0.1,name='incresB_2')
    x = incresB(x,0.1,name='incresB_3')
#     x = incresB(x,0.1,name='incresB_4')
#     x = incresB(x,0.1,name='incresB_5')

    #17 × 17 to 8 × 8 reduction module.
    x_red_21 = MaxPool2D(3,strides=2,padding='valid',name='red_maxpool_2')(x)

    x_red_22 = conv2d(x,64,1,1,'same',True,name='x_red2_c11')
    x_red_22 = conv2d(x_red_22,96,3,2,'valid',True,name='x_red2_c12')

    x_red_23 = conv2d(x,64,1,1,'same',True,name='x_red2_c21')
    x_red_23 = conv2d(x_red_23,64,3,2,'valid',True,name='x_red2_c22')

    x_red_24 = conv2d(x,64,1,1,'same',True,name='x_red2_c31')
    x_red_24 = conv2d(x_red_24,64,3,1,'same',True,name='x_red2_c32')
    x_red_24 = conv2d(x_red_24,64,3,2,'valid',True,name='x_red2_c33')

    x = Concatenate(axis=3, name='red_concat_2')([x_red_21,x_red_22,x_red_23,x_red_24])

    #Inception-ResNet-C modules
    x = incresC(x,0.2,name='incresC_1')
    x = incresC(x,0.2,name='incresC_2')
    x = incresC(x,0.2,name='incresC_3')
    #TOP
    x = GlobalAveragePooling2D(data_format='channels_last')(x)
    x = Dropout(0.6)(x)
    z = Dense(n_classes, activation='softmax')(x)
    
    model = Model(ip,z)
#     plot_model(model = model, to_file = './img/InceptionResNet.png', show_layer_names=False, show_shapes=True)
    model.summary()
    
    return model

## Compile Model

In [20]:
VOCAB_SIZE = sp.GetPieceSize()+1 #embedding_matrix.shape[0]
MAXSEQLEN = X_toks_train.shape[1]
# MAXCHARSLEN = X_train.shape[1]
REPEAT=5
EMBVECSIZE=300

model = get_model(vocab_size = VOCAB_SIZE,
                  emb_vec_size=EMBVECSIZE,
                  repeat=REPEAT,
                  n_classes=len(chars), 
                  seq_len_emb=MAXSEQLEN, 
                  seq_len_chars=None)

optimizer = Adam(lr=1e-3, decay=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input1 (InputLayer)             (None, 45)           0                                            
__________________________________________________________________________________________________
BPE_emb (Embedding)             (None, 45, 300)      45300       Input1[0][0]                     
__________________________________________________________________________________________________
repeat_elems (Lambda)           (None, 45, 1500)     0           BPE_emb[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 45, 300, 5)   0           repeat_elems[0][0]               
__________________________________________________________________________________________________
conv1conv2

### If we wanna use Embedding Matrix with FT vectors then one should uncomment the following lines-->

In [ ]:
embedding = model.get_layer('BPE_emb')
embedding.set_weights([embedding_matrix])
embedding.trainable = False

## Continue Training: Load Model from the last Checkpoint

In [21]:
path = './models/keras/inseption_resnet_emb_fasttext.h5'
model = load_model(path)

In [25]:
optimizer = Adam(lr=1e-4, decay=0.)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

## Run Training

In [22]:
path = './models/keras/inseption_resnet_emb_fasttext.h5'

callbacks = [EarlyStopping(patience=12, monitor='acc'),
             ModelCheckpoint(path, monitor='acc', verbose=1, save_best_only=True),
             ReduceLROnPlateau(monitor='acc', factor=1e-5, patience=3, verbose=1)]

BATCH_SIZE=256
n_epoch=200

history = model.fit(X_toks_train, y_train, batch_size=BATCH_SIZE, epochs=n_epoch, shuffle=True,
                    validation_split=0.15, verbose=1, callbacks=callbacks).history

Train on 2166337 samples, validate on 382295 samples
Epoch 1/200
  57600/2166337 [..............................] - ETA: 1:40:04 - loss: 1.3455 - acc: 0.3389

KeyboardInterrupt: 

In [ ]:
plot_metrics(history)

## OOB Validation

In [ ]:
model = load_model(path)

In [ ]:
y_pred = model.predict([X_toks_test, X_test])
y_pred = y_pred.argmax(axis=-1)

y_true = y_test.argmax(axis=-1)

In [ ]:
print(classification_report(y_true, y_pred))